In [24]:
import json
import os
from collections import defaultdict
os.environ['ALFRED_ROOT'] = '/root/data/home/hoyeung/alfred'

In [25]:
with open('../../../../../data_alfred/splits/collect_states_20200511_valid_seen_notebook_success_paths.json', 'r') as f:
    valid_seen_object_state_collection_success_paths = json.load(f)
valid_seen_tasks = [p.split('/')[4] for p in valid_seen_object_state_collection_success_paths]
valid_seen_trajs = [p.split('/')[5] for p in valid_seen_object_state_collection_success_paths]

with open('../../../../../data_alfred/splits/collect_states_20200511_valid_unseen_notebook_success_paths.json', 'r') as f:
    valid_unseen_object_state_collection_success_paths = json.load(f)
valid_unseen_tasks = [p.split('/')[4] for p in valid_unseen_object_state_collection_success_paths]
valid_unseen_trajs = [p.split('/')[5] for p in valid_unseen_object_state_collection_success_paths]

In [26]:
skill_set = {0:'Random Choice'}
skill2num = {'Random Choice':0}
pickupObject_set={0:'Random Choice'}
pickupObject2num = {'Random Choice':0}
movable_set = {0:'Random Choice'}
movable2num = {'Random Choice':0}
receptacle_set = {0:'Random Choice'}
receptacle2num = {'Random Choice':0}
lookup = defaultdict(lambda:defaultdict(lambda:defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(list))))))

def add_to_set(val, val_set, val2num):
    if val not in val_set.values():
        idx = len(val_set)
        val_set[idx] = val
        val2num[val] = idx

def process_tasks(tasks, split, success_traj_ids):
    for task in tasks:
        skill, pickupObject, movable, receptacle, scene_num = task.split('-')
        add_to_set(skill, skill_set, skill2num)
        add_to_set(pickupObject, pickupObject_set, pickupObject2num)
        add_to_set(movable, movable_set, movable2num)
        add_to_set(receptacle, receptacle_set, receptacle2num)
        traj_ids = os.listdir('../../../../../data_alfred/json_feat_2.1.0/{}/{}'.format(split, task))
        # TODO filter down to only valid traj_ids
        traj_ids = [traj_id for traj_id in traj_ids if traj_id in success_traj_ids]

        lookup[skill2num[skill]][pickupObject2num[pickupObject]][movable2num[movable]][receptacle2num[receptacle]][int(scene_num)][split] += traj_ids

In [27]:
process_tasks(valid_unseen_tasks, 'valid_unseen', valid_unseen_trajs)
process_tasks(valid_seen_tasks,  'valid_seen', valid_seen_trajs)

In [29]:
with open('task_tuple_lookup.json', 'w') as f:
    json.dump(lookup, f)
with open('task_tuple_lookup.json', 'r') as f:
    lookup = json.load(f)

In [30]:
movable_set

{0: 'Random Choice',
 1: 'None',
 2: 'Mug',
 3: 'Pan',
 4: 'Box',
 5: 'Plate',
 6: 'Cup',
 7: 'Bowl',
 8: 'Pot'}

In [31]:
import pickle

save = {
'skill_set':skill_set, 
'skill2num':skill2num, 
'pickupObject_set':pickupObject_set, 
'pickupObject2num':pickupObject2num,
'movable_set':movable_set,
'movable2num':movable2num,
'receptacle_set':receptacle_set,
'receptacle2num':receptacle2num,
'lookup':lookup
}

pickle.dump( save, open( "task_lookup.p", "wb" ) )
save = pickle.load( open( "task_lookup.p", "rb" ) )